In [ ]:
%%html
<style>
.output_subarea.output_text.output_stream.output_stdout > pre {
    width:max-content;
}
.p-Widget.jp-RenderedText.jp-OutputArea-output > pre {
   width:max-content;
}
</style>

In [ ]:
import pandas as pd
import numpy as np
import scipy.optimize as sco
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.sql import SparkSession, Window, Row
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import lit, year, month, dayofmonth, last_day, col, array, explode, struct, udf, to_date, month, year, percent_rank, when
from pyspark.sql.functions import sum as spark_sum, max as spark_max

In [ ]:
spark = SparkSession.builder.appName('Rank_Median').getOrCreate()
sc = spark.sparkContext
sc

In [ ]:
## por temas de replicación, siempre es mejor convertir el archivo a leer en formato csv

monthly_data = spark.read.parquet("data/master/ophelia/data/OpheliaData/analytical_base_table")
index_vector = spark.read.csv("data/raw/csv/unique_dateprice_vector.csv", header=True, inferSchema=True)

In [ ]:
from datetime import datetime

def datetime_object(datetime_str):
    print(datetime_str)
    datetime_object = datetime.strptime(datetime_str, '%d/%m/%y').date()
    return datetime_object

datetime_object_udf = udf(f=datetime_object, returnType=DateType())

In [ ]:
index_vector_schema = index_vector.select("MXWDU_Index", datetime_object_udf(col("operation_date")).alias('operation_date'))
index_vector_schema.show(10, False)

In [ ]:
close_date_index = index_vector_schema.select("*", month(col("operation_date")).alias("month"), year(col("operation_date")).alias("year"))\
                                      .groupBy("year", "month").agg(spark_max("operation_date").alias("operation_date")).drop("year", "month")\
                                      .join(index_vector_schema, on="operation_date", how="left")
close_date_index.orderBy(col("operation_date").desc()).show(10, False)

In [ ]:
index_portfolio_df = close_date_index.join(monthly_data, on="operation_date", how="left")
index_portfolio_df.orderBy(col("operation_date").desc()).show(20, False)

In [ ]:
plt.style.use('fivethirtyeight')
np.random.seed(777)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
def evolved_time_frame(df):
    table = df.toPandas()
    plt.figure(figsize=(14, 7))
    for c in table.columns.values:
        plt.plot(table.index, table[c], lw=3, alpha=0.8,label=c)
    #plt.legend(loc='upper left', fontsize=12)
    plt.ylabel('price in $')

In [ ]:
evolved_time_frame(index_portfolio_df)

In [ ]:
monthly_data = index_portfolio_df.toPandas()
print("nuestros datos:", monthly_data.shape)

In [ ]:
## seleccionamos la primera columna y lo convertimos a vector, esta columna representa el indice que estamos analizando, 
## 'MXWDU_Index', la idea es medir esta columna con el resto, ya que las demás son acciones que forman parte de ése índice.

benchmark_month = monthly_data["MXWDU_Index"]

# nuestros datos
print(benchmark_month.shape)

In [ ]:
## Calculamos el porcentaje de cambio de un día contra otro.
## fórmula: (t+1 / t)-1
## pseudo-código: (precio_hoy / precio_ayer)-1

pct_benchmark_month = benchmark_month.pct_change(1)
print(pct_benchmark_month.shape)

# nuestros datos

In [ ]:
## el vector de "percentage change" se convierte a un arreglo numpy de dimensión (147,)
## NOTA: en los arreglos (objetos) de tipo numpy.array preservan los valores, tanto por fila, como por columna,
##       el órden de los elementos, cómo un 'índice implícito'

pct_benchmark_month_array = np.array(pct_benchmark_month)
print(pct_benchmark_month_array.shape)

In [ ]:
## Lo mismo hacemos, pero para el resto de variables equity,
## seleccionamos la primera columna y lo convertimos a vector, esta columna representa el indice que estamos analizando, 
## 'MXWDU_Index', la idea es medir esta columna con el resto, ya que las demás son acciones que forman parte de ese índice.

investment_universe_month = monthly_data.drop(['operation_id', "operation_date", "MXWDU_Index"], axis = 1)

# nuestros datos
investment_universe_month.shape

In [ ]:
## (t+1 / t)-1
## (precio_hoy / precio_ayer)-1

pct_investment_month = investment_universe_month.pct_change(1)

# nuestros datos
pct_investment_month.shape

In [ ]:
## el vector de percentage change se convierte a un arreglo numpy de dimensión (147,70)

pct_investment_month_array = np.array(pct_investment_month)
#_pct_investment_month_array = np.array(_pct_investment_month)

# nuestros datos
print(pct_investment_month_array.shape)

In [ ]:
## creamos arreglos numpy con dimensiones X+1 = 148, rellenas de ceros, para ser imputados con nuevos vectores

up_month = np.zeros((pct_benchmark_month_array.shape[0]+1, 1))
down_month = np.zeros((pct_benchmark_month_array.shape[0]+1, 1))
up_move = np.zeros((pct_benchmark_month_array.shape[0]+1, pct_investment_month_array.shape[1]))
down_move = np.zeros((pct_benchmark_month_array.shape[0]+1, pct_investment_month_array.shape[1]))

print(up_month.shape)
print(down_month.shape)
print(up_move.shape)
print(down_move.shape)

In [ ]:
## rellenamos las matrices de ceros con valores que aprueben las condiciones, 
## se realiza una comparación dentro de los arreglos de porcentajes de cambio, 
## sí alguno de esos porcentajes es superior a 0, entonces entra a los arreglos 
## de movimientos positivos (incrementos), pero sí alguno es menor que 0, entonces
## el porcentaje se almacena en los arreglos de movimientos negativos (decrementos).

## Básicamente, se separan los porcentajes de cambio en dos matrices: 
## matriz de positivos cuando el porcentaje es > 0 
## matriz de negativos cuando el porcentaje es <= 0

size_benchmark_matrix = pct_benchmark_month_array.shape[0]
for i in range (1, size_benchmark_matrix):
    if pct_benchmark_month_array[i] > 0:
        up_month[i] = pct_benchmark_month_array[i]
        up_move[i] = pct_investment_month_array[i, 0:pct_investment_month_array.shape[1]]
    else:
        down_month[i] = pct_benchmark_month_array[i]
        down_move[i] = pct_investment_month_array[i, 0:pct_investment_month_array.shape[1]]

In [ ]:
## calculamos los vectores 'peor más alto' y 'mejor más alto'

np.seterr(divide='ignore', invalid='ignore')
greater_worse = down_move / down_month
greater_better = (up_move / up_month) * float(-1.0)

print(greater_worse.shape)
print(greater_better.shape)

In [ ]:
## ambos vectores los convertimos a pandas dataframes, y solo nos quedamos con los vectores que tengan valores != np.nan
## una de las ventajas de los pandas dataframes es que mantienen un ídince único por row, esto lo hace poder separarse, y juntarse
## en cuantos sub-conjuntos se requieran y siempre se podrá mantener un órden.

greater_worse_df = pd.DataFrame(data=greater_worse).dropna()
greater_better_df = pd.DataFrame(data=greater_better).dropna()

print(greater_worse_df.shape)
print(greater_better_df.shape)

In [ ]:
## calculamos ahora, la mediana acumulada con los pandas dataframes que construimos, 
## con un periodo mínimo (método expanding) de al menos 1 observación dada.

median_down = greater_worse_df.expanding().median()
median_up = greater_better_df.expanding().median()

print(median_down.shape)
print(median_up.shape)

In [ ]:
# se transponen ambos pandas df por la columna periodos, columna que almacena números no consecutivos desde 1 hasta 147

down_transpose = median_down.T
up_transpose = median_up.T

print(down_transpose.shape)
print(up_transpose.shape)

In [ ]:
# se rankean los resultados (top 10) entre las fechas cierre (periodo) y se vuelve a transponer la tabla ranked_down

ranked_down = down_transpose.rank(method='first')
transpose_ranked_down = ranked_down.T

print(ranked_down.shape)
print(transpose_ranked_down.shape)

In [ ]:
# se rankean los resultados (top 10) entre las fechas cierre (periodo) y se vuelve a transponer la tabla ranked_up

ranked_up = up_transpose.rank(method='first')
transpose_ranked_up = ranked_up.T

print(ranked_up.shape)
print(transpose_ranked_up.shape)

In [ ]:
## se añade variable 'label' con la idea de que al juntar ambos dataframes se puedan distinguir los 'worse' de los 'better'
## y se unen ambos dataframes con la etiqueta creada, se usó el método 'insert' por lo que no se deberá correr de nuevo, una
## vez ejecutado ya que fallará por duplicidad de columnas.

worse_better_df = pd.concat([transpose_ranked_up, transpose_ranked_down]).sort_index()
worse_better_df.shape

In [ ]:
## se crea un índice 'closing_id' para cada registro, éste corre de [1:N+1] 
## con la idea de etiquetar el id del mes de registro de cierre,
## de la misma forma que lo anterior, NO se deberá ejecutar de nuevo; una vez hecho.

worse_better_df['closing_id'] = range(1, len(worse_better_df) + 1)
worse_better_df.shape

In [ ]:
## se transponen ambos dataframes, de antes tener una dimensión (68, 70), es decir; 
## 68 registros i.e. 'Rows' (variables)
## 70 columnas fijas (a menos que sea añadido otro asset desde el csv inicial)

## a tener una dimensión 'transpuesta' (invertída sí querés...) de (70, 68), es decir;
## 70 registros i.e. 'Rows' fijos (a menos que sea añadido otro asset desde el csv inicial)
## 68 columnas (variables)

In [ ]:
## de pandas dataframes, una vez separados en dos conjuntos ['worse', 'better'],
## creamos por separado dos spark dataframes.

worse_better = spark.createDataFrame(worse_better_df)
worse_better.show(5, False)

In [ ]:
worse_better.coalesce(1).write.mode("overwrite").parquet("data/ophelia/out/engine/RankMedian")

# Finish Rank Median Module

# Start TopAssetRank

In [ ]:
worse_better = spark.read.parquet("data/ophelia/out/engine/RankMedian/")

In [ ]:
## se crea un generador "shape long format", una lista con iteraciones, esta lista trabajará con dos variables principales,
## 1-. la variable 'equity_index', será la que contenga los 'id' de los activos
## 2-. la variable 'median_down', será la que contenga la mediana acumulada por cada activo.
## Se mantendrá a lo largo de la transformación 1 columna fija; 'closing_id',
## closing_id: variable que indica el mes de cierre y reporte de precio

def panel_format(df, pivot_col, new_columns: list = []):
    first_col = str(new_columns[0])
    second_col = str(new_columns[1])
    piv_col = [pivot_col]
    df_types = df.dtypes
    cols, dtype = zip(*((c, t) for (c, t) in df_types if c not in piv_col))
    if len(set(dtype)) == 1:
        ValueError("Columns not the same data type...")
    generator_explode = explode(array([
        struct(lit(c).alias(first_col), col(c).alias(second_col)) for c in cols
    ])).alias("column_explode")
    column_to_explode = ["column_explode."+first_col, "column_explode."+second_col]
    panel_df = df.select(piv_col + [generator_explode])\
                 .select(piv_col + column_to_explode)
    return panel_df

In [ ]:
## se crea nuevo spark-dataframe donde solo se mostrará por partición ['closing_id'] 
## el top 10 mejores meses donde tuvo menos malos que el resto de los registros; ["top_rank"].
new_columns = ["asset_id", "rank"]
asset_ranking_df = panel_format(worse_better, "closing_id", new_columns)
asset_ranking_df.printSchema()
asset_ranking_df.show(10, False)

In [ ]:
asset_ranking_df.coalesce(1).write.mode("overwrite").parquet("data/ophelia/out/engine/LongMedianRank/")

# Finish LongMedianRank

# Start TrainPortfolioSimulation

In [ ]:
asset_ranking_df = spark.read.parquet("data/ophelia/out/engine/LongMedianRank/")
asset_rank_10 = asset_ranking_df.where(col("rank") <= 10).orderBy("closing_id", "rank").toPandas()
asset_rank_10

In [ ]:
## lo que se pretende ahora, es obtener una matriz de dimensión (146, 10), es decir, 
## tener en cada row las fechas de cierre [closing_id],
## en cada columna (header) el número de ranking top 10 [top_rank],
## y en cada campo, el id del activo [asset_id].

pandas_df = asset_rank_10.astype({'asset_id':'int32'})
newselect = asset_rank_10[["closing_id", "asset_id"]]
num_of_assets = 10

indexed = np.zeros((investment_universe_month.shape[0], num_of_assets))

for q in range(1,investment_universe_month.shape[0]):
    selection = newselect.loc[pandas_df["closing_id"]==q]
    newselect_transpose = selection.T
    newpdf = newselect_transpose['asset_id':].head()
    indexed[q] = newpdf

In [ ]:
indexed = indexed[1:investment_universe_month.shape[0]+1]
index_row = indexed.astype(np.int64)
newrow = [0]*num_of_assets #np.zeros((1,num_of_assets))
index_row = np.vstack([index_row,newrow])
portfolio = np.zeros((investment_universe_month.shape[0], num_of_assets))

for r in range(0,investment_universe_month.shape[0]-1):
    s = r+1
    columns = index_row[s]
    portfolio[s] = pct_investment_month_array[s,[columns]]

In [ ]:
means = portfolio[1:investment_universe_month.shape[0]-1]
performance = np.dot(portfolio,(1/num_of_assets))
returns = np.zeros((investment_universe_month.shape[0]-1,1))
equalw = np.zeros((investment_universe_month.shape[0]-1,1))
eweights = 1/(pct_investment_month_array.shape[1])
eweighted = np.dot(pct_investment_month_array,eweights)

for x in range (1,investment_universe_month.shape[0]):
    returns[x-1] = sum(performance[x])
    equalw[x-1] = sum(eweighted[x])

In [ ]:
beg = 100
start = 100
commence = 100
bench = pct_benchmark_month_array[1:investment_universe_month.shape[0]]
bmk = np.zeros((investment_universe_month.shape[0],))
port = np.zeros((investment_universe_month.shape[0]-1,))
ew = np.zeros((investment_universe_month.shape[0]-1,))

for i in range (0,investment_universe_month.shape[0]-1):
    bmk[i] = beg*(1+bench[i])
    beg = bmk[i]
    port[i] = start*(1+returns[i])
    start = port[i]
    ew[i] = commence*(1+equalw[i])
    commence = ew[i]

plt.plot(bmk[0:investment_universe_month.shape[0]-1])
plt.plot(port)
plt.plot(ew)

In [ ]:
input_assets = index_row[index_row.shape[0]-2]
input_assets1 = pct_investment_month_array[:, input_assets]
dataframe = pd.DataFrame(input_assets1)
input_assets1 = dataframe.dropna()
mean_returns = input_assets1.mean()
covar_matrix = input_assets1.cov()
tickers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

def calc_portfolio_perf(weights, mean_returns, cov, rf):
    portfolio_return = np.sum(mean_returns * weights) * 12
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(12)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    return portfolio_return, portfolio_std, sharpe_ratio

In [ ]:
def simulate_random_portfolios(num_portfolios, mean_returns, cov, rf):
    results_matrix = np.zeros((len(mean_returns)+3, num_portfolios))
    for i in range(num_portfolios):
        weights = np.random.random(len(mean_returns))
        weights /= np.sum(weights)
        portfolio_return, portfolio_std, sharpe_ratio = calc_portfolio_perf(weights, mean_returns, cov, rf)
        results_matrix[0,i] = portfolio_return
        results_matrix[1,i] = portfolio_std
        results_matrix[2,i] = sharpe_ratio

        for j in range(len(weights)):
            results_matrix[j+3,i] = weights[j]

    results_df = pd.DataFrame(results_matrix.T, columns=['ret','stdev','sharpe'] + [ticker for ticker in tickers])

    return results_df

In [ ]:
# TODO: revisar la sobrecarga de memoria en los stages marcados

mean_returns = input_assets1.mean()
cov = input_assets1.cov()
num_portfolios = 10000000
rf = 0.0

results_frame = simulate_random_portfolios(num_portfolios, mean_returns, cov, rf)

max_sharpe_port = results_frame.iloc[results_frame['sharpe'].idxmax()]

min_vol_port = results_frame.iloc[results_frame['stdev'].idxmin()]

max_return_port = results_frame.iloc[results_frame['ret'].idxmax()]

plt.subplots(figsize=(15,10))
plt.scatter(results_frame.stdev, results_frame.ret, c=results_frame.sharpe, cmap='RdYlBu')
plt.xlabel('Standard Deviation')
plt.ylabel('Returns')
plt.colorbar()
plt.scatter(max_sharpe_port['stdev'], max_sharpe_port['ret'], marker=(5,1,0), color='r', s=500)
plt.scatter(min_vol_port['stdev'], min_vol_port['ret'], marker=(5,1,0), color='g', s=500)
plt.scatter(max_return_port['stdev'], max_return_port['ret'], marker=(5,1,0), color='y', s=500)
plt.show()

In [ ]:
renamed_columns = {
    0: 'fund_w_zero',
    1: 'fund_w_one',
    2: 'fund_w_two',
    3: 'fund_w_three',
    4: 'fund_w_four',
    5: 'fund_w_five',
    6: 'fund_w_six',
    7: 'fund_w_seven',
    8: 'fund_w_eight',
    9: 'fund_w_nine'
}
results_frame_renamed = results_frame.rename(columns=renamed_columns)
results_frame_renamed.to_csv('data/ophelia/out/model/tmp/TrainSimulation.csv', encoding='utf-8', index=False)
results_frame_renamed

In [ ]:
def pd_series_to_dict(pd_series):
    to_python_dict = dict(pd_series)
    return {str(k): float(v) for k, v in to_python_dict.items()}

In [ ]:
def key_map_rename(dic, new_key, old_key):
    for key in range(len(old_key)):
        dic[new_key[key]] = dic.pop(old_key[key])
    return dic

In [ ]:
def persist_metadata_model(dic_sample, json_name):
    dict_to_json = spark.read.json(sc.parallelize([dic_sample]))
    dict_to_json.coalesce(1).write.mode("overwrite").json("data/ophelia/out/model/model_info/"+str(json_name))

In [ ]:
max_return_port_sample = pd_series_to_dict(max_return_port)
persist_metadata_model(max_return_port_sample, "maxReturnTrainWeights")

In [ ]:
max_sharpe_port_sample = pd_series_to_dict(max_sharpe_port)
persist_metadata_model(max_sharpe_port_sample, "maxSharpeTrainWeights")

In [ ]:
min_vol_port_sample = pd_series_to_dict(min_vol_port)
persist_metadata_model(min_vol_port_sample, "minVolatileTrainWeights")

In [ ]:
#tmp_read.coalesce(1).write.mode("overwrite").parquet("data/ophelia/out/model/TrainSimulation/")
#tmp_read.show(5, False)

# Finish TrainPortfolioSimulation

# Start EfficientFrontier

In [ ]:
#key_o = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
#key_n = ['fund_w_zero', 'fund_w_one', 'fund_w_two', 'fund_w_three', 'fund_w_four', 
#         'fund_w_five', 'fund_w_six', 'fund_w_seven', 'fund_w_eight', 'fund_w_nine']
#
#min_vol_port_sample = key_map_rename(pd_series_to_dict(min_vol_port), key_n, key_o)
#min_vol_port_sample

In [ ]:
results_spark_df = spark.read.csv("data/ophelia/out/model/tmp/TrainSimulation.csv", header='true')
results_spark_df.show(5, False)

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id, current_date, current_timestamp

results_index = results_spark_df.select("*", (monotonically_increasing_id() + 1000).alias("id"), current_date().alias("information_date"), current_timestamp().alias("model_date"))
results_index.show(5, False)

In [ ]:
percentile_bucketizer = results_index.select("*", percent_rank().over(Window.orderBy(results_index['sharpe'])).alias("percentile"))\
                                     .select("*", 
                                             when(col("percentile") < 0.2, lit(5.0)).otherwise(
                                                 when((col("percentile") >= 0.2) & (col("percentile") < 0.4), lit(4.0)).otherwise(
                                                     when((col("percentile") >= 0.4) & (col("percentile") < 0.6), lit(3.0)).otherwise(
                                                         when((col("percentile") >= 0.6) & (col("percentile") < 0.8), lit(2.0)).otherwise(
                                                             when((col("percentile") >= 0.8) & (col("percentile") <= 1.0), lit(1.0)))))).alias("bucket"))
percentile_bucketizer.show(10, False)

In [ ]:
def filter_min_return(df, dic):
    for col in min_vol_port_sample:
        filter_df = df.filter(df[col] >= dic[col])
    return filter_df

In [ ]:
fiter_min_return_df = filter_min_return(percentile_bucketizer, min_vol_port_sample)

In [ ]:
fiter_min_return_df.groupBy("bucket").count().show()

In [ ]:
centile_bucketizer.coalesce(1).write.mode("overwrite").parquet('data/ophelia/out/model/RandSimulation/', partitionBy="information_date")

In [ ]:
# testing

# cual es el mejor port con base en el sharpe para cada cliente

sorted_results = centile_bucketizer.orderBy(col("sharpe").desc())
sorted_results.show(10, False)
pd_results = sorted_results.toPandas()

In [ ]:
def get_ret_vol_sr(weights):
    weights = np.array(weights)
    ret = np.sum(mean_returns * weights) * 12
    vol = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(12)
    sr = ret/vol
    return np.array([ret, vol, sr])

def neg_sharpe(weights):
    # the number 2 is the sharpe ratio index from the get_ret_vol_sr
    return get_ret_vol_sr(weights)[2] * -1

def check_sum(weights):
    #return 0 if sum of the weights is 1
    return np.sum(weights)-1

cons = ({'type':'eq', 'fun':check_sum})
bounds = ((0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1))
init_guess = [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1]
opt_results = sco.minimize(neg_sharpe, init_guess, method='SLSQP', bounds=bounds, constraints=cons)
print("opt_results:", opt_results)

get_ret_vol_sr(opt_results.x)

vol_arr = results_frame.stdev
ret_arr = results_frame.ret
sharpe_arr = results_frame.sharpe

range_vol_min = min(vol_arr)
range_vol_max = max(vol_arr)
range_ret_min = min(ret_arr)
range_ret_max = max(ret_arr)
range_sharpe_min = min(sharpe_arr)
range_sharpe_max = max(sharpe_arr)

frontier_y = np.linspace(range_ret_min, range_ret_max, 200)
frontier_x = []


def minimize_volatility(weights):
    return get_ret_vol_sr(weights)[1]

for possible_return in frontier_y:
    cons = ({'type':'eq', 'fun': check_sum},
            {'type':'eq', 'fun': lambda w: get_ret_vol_sr(w)[0] - possible_return})
    result = sco.minimize(minimize_volatility, init_guess, method='SLSQP', bounds=bounds, constraints=cons)
    frontier_x.append(result['fun'])


# cmap='RdYlBu' 

plt.figure(figsize=(12,8))
plt.scatter(vol_arr, ret_arr, c=sharpe_arr, cmap='RdYlBu')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')
plt.plot(frontier_x, frontier_y, 'r--', linewidth=3)
# plt.savefig('cover.png')
plt.show()